In [1]:
import os
import json
import pickle
import shutil
import pandas as pd

import tensorflow as tf

from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2

from google.protobuf import text_format

In [2]:
# Set up labels and pre trained model

In [3]:
labels = [{'name':'Car',
           'id':1
          },
          {'name':'Motorbike',
           'id':2
          }]

In [4]:
with open('../config/label_map.pbxt','w') as f:
    for label in labels:
        f.write('item { \n')
        f.write(f"\tname:\'{label['name']}\'\n")
        f.write(f"\tid:{label['id']}\n")
        f.write('}\n')

In [26]:

# Mobilenet model dowloaded from terminal using:  

# wget --no-check-certificate http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz -O ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

# see docs : https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md
mobilenet_dir = '../ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'



In [27]:
# config file cominig from pretrained architecture
# config pipeline file has all the configuration parameters that the model will use while learning
# params like batch size, image reshape, data augmentation config, regularization etc

In [45]:
#checkpoints will be saved here and pipeline config file
path_training = '../custom_ssd_mobilenet'

In [30]:
source_config = f"{mobilenet_dir}/pipeline.config"
target_config = f"../{path_training}/pipeline.config"
shutil.copyfile(source_config, target_config)

'../custom_ssd_mobilenet/pipeline.config'

In [31]:
# Set up config file to make it editable

In [32]:
config = config_util.get_configs_from_pipeline_file(target_config)

In [33]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(target_config,"r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [34]:
label_map_pbxt_fname = '../config/label_map.pbxt'
train_record_path = '../labels/train.record'
test_record_path = '../labels/test.record'

In [35]:
# EDIT THE PIPELINE.CONFIG FILE
pipeline_config.model.ssd.num_classes = 2
pipeline_config.train_config.batch_size = 8
pipeline_config.train_config.fine_tune_checkpoint = f"{mobilenet_dir}/checkpoint/ckpt-0"
pipeline_config.train_config.fine_tune_checkpoint_type = 'detection'

pipeline_config.train_input_reader.label_map_path = label_map_pbxt_fname
pipeline_config.train_input_reader.tf_record_input_reader.input_path[0] = train_record_path

pipeline_config.eval_input_reader[0].label_map_path = label_map_pbxt_fname
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[0] = test_record_path


In [36]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(target_config, "wb") as f:
    f.write(config_text)

In [41]:
path_training

'custom_ssd_mobilenet'

In [39]:
num_steps = 500
model_dir = path_training

!python3 ../models/research/object_detection/model_main_tf2.py\
--pipeline_config_path={target_config} \
--model_dir={model_dir} \
--num_training_steps={num_steps}

2022-09-29 23:26:00.174566: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-29 23:26:00.181292: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-29 23:26:00.181938: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-29 23:26:00.182659: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [46]:
%load_ext tensorboard
%tensorboard --logdir "../custom_ssd_mobilenet"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 113643), started 0:05:25 ago. (Use '!kill 113643' to kill it.)

In [47]:
output_dir = '../tuned_model'
!python3 ../models/research/object_detection/exporter_main_v2.py \
--input_type image_tensor \
--pipeline_config_path {target_config} \
--trained_checkpoint_dir {path_training} \
--output_directory {output_dir}

2022-09-29 23:56:52.046292: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-29 23:56:52.053143: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-29 23:56:52.053579: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-29 23:56:52.058108: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [48]:
!zip -r fined_tuned_model.zip ../tuned_model

  adding: ../tuned_model/ (stored 0%)
  adding: ../tuned_model/checkpoint/ (stored 0%)
  adding: ../tuned_model/checkpoint/ckpt-0.data-00000-of-00001 (deflated 8%)
  adding: ../tuned_model/checkpoint/ckpt-0.index (deflated 80%)
  adding: ../tuned_model/checkpoint/checkpoint (deflated 39%)
  adding: ../tuned_model/pipeline.config (deflated 68%)
  adding: ../tuned_model/saved_model/ (stored 0%)
  adding: ../tuned_model/saved_model/saved_model.pb (deflated 92%)
  adding: ../tuned_model/saved_model/variables/ (stored 0%)
  adding: ../tuned_model/saved_model/variables/variables.index (deflated 78%)
  adding: ../tuned_model/saved_model/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: ../tuned_model/saved_model/assets/ (stored 0%)


In [49]:
# upload to tensorboard web page

In [50]:
# cli command
# !tensorboard dev upload --logdir "ssd_mobilenet" --name 'cars motorbike object detection' --description 'training using ssd + mobilenet' --one_shot

In [51]:
# Published here!!
#https://tensorboard.dev/experiment/wQY50j2JRdSJDdB4vIQOsw/